In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def fetch_page(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

def get_genres(source_url):
    soup = fetch_page(source_url)
    genres_list = []
    get_genres = soup.find('ul', class_='cnt-list')
    for li in get_genres.find_all('li'):
        a_tag = li.find('a')
        href = a_tag['href'].replace('/estilos/', '').rstrip('/')
        genres_list.append(href)
    return genres_list

def get_top10_songs(genre_urls, base_url_genres):
    top10_songs = []
    for genre_url in genre_urls:
        soup = fetch_page(genre_url)
        get_top10 = soup.find('ol', class_='top-list_mus')
        if get_top10:
            for li in get_top10.find_all('li'):
                tag = li.find('a')
                if tag and tag.has_attr('href'):
                    top10_songs.append(base_url_genres + tag['href'])
    return top10_songs

def fetch_song_details(song_url):
    soup = fetch_page(song_url)
    achar_letra = soup.find('div', class_='lyric-original')       
    letra = achar_letra.find_all('p')
    letra_limpa = ''.join([p.text for p in letra])
    
    achar_artista_e_musica = soup.find('div', class_='title-content')  
    artista = achar_artista_e_musica.find('h2').text.strip()
    musica = achar_artista_e_musica.find('h1').text.strip()

    letra_limpa_final = re.sub(r'([;áéíóúàèìòùãõâêîôû;a-z,;!;.;?;)])([;A-Z;ÁÉÍÓÚÂÊÎÔÛÃÕ;])', r'\1\n\2', letra_limpa)
    return artista, musica, letra_limpa_final

def create_lyrics_database(genres_list, top10_songs):
    Artista = []
    Musica = []
    Letra = []

    for song in top10_songs:
        artista, musica, letra_limpa_final = fetch_song_details(song)
        Musica.append(musica)
        Artista.append(artista)
        Letra.append(letra_limpa_final)

    dict_final = {}
    num_musicas_por_genero = 10
    inicio = 0
    i = 0
    fim = num_musicas_por_genero

    for genero in genres_list:
        dict_final[genero] = {
            'Artista': Artista[inicio:fim],
            'Musica': Musica[inicio:fim],
            'Letra': Letra[inicio:fim]
        }
        i += 1
        inicio = i * num_musicas_por_genero
        fim = inicio + num_musicas_por_genero

    df = pd.DataFrame.from_dict(dict_final, orient='index')
    df = df.apply(pd.Series.explode).reset_index()
    return df

def adicionar_quebra_de_linha(letra):
    padrao = r'([a-záéíóúàèìòùãõâêîôû;!?.])([A-ZÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛ;¿¡])'
    return re.sub(padrao, r'\1\n\2', letra)

def main():
    source_url = 'https://www.letras.mus.br/estilos/'
    base_url = 'https://www.letras.mus.br/estilos/'
    base_url_genres = 'http://www.letras.mus.br'

    genres_list = get_genres(source_url)
    genre_urls = [base_url + genre for genre in genres_list]
    top10_songs = get_top10_songs(genre_urls, base_url_genres)
    df = create_lyrics_database(genres_list, top10_songs)

    df['Letra limpa'] = df['Letra'].apply(adicionar_quebra_de_linha)
    df.to_csv('lyrical_database.csv', index=False)

if __name__ == "__main__":
    main()


In [4]:
import pandas as pd
import re
from collections import Counter

df = pd.read_csv('lyrical_database.csv')
df
filtro = df['Letra'].isnull()

df[filtro]
df[filtro].index
df.drop(df[filtro].index, inplace = True)

# Function to calculate lexical diversity
def lexical_diversity(text):
    words = text.split()
    return len(set(words)) / len(words)



# Funcao para processar as letras
def preprocess_lyrics(lyrics):
    # Normalize line breaks and whitespace
    lyrics = re.sub(r'\s+', ' ', lyrics).strip()
    # Convert to lowercase
    lyrics = lyrics.lower()
    # Remove punctuation
    lyrics = re.sub(r'[^\w\s]', '', lyrics)
    return lyrics

df['Letra limpa'] = df['Letra limpa'].astype("string")
# Aplicar a funcao para as letras
df['Letras processadas'] = df['Letra limpa'].apply(preprocess_lyrics)

# Mostrando as letras processadas para vizualizacao
df[['Artista', 'Musica', 'Letras processadas']].head()

# Calculate vocabulary size and lexical diversity for each song
df['Volume de Vocabulario'] = df['Letras processadas'].apply(lambda x: len(set(x.split())))
df['Diversidade Lexical'] = df['Letras processadas'].apply(lexical_diversity)

# Display the results
df[['Artista', 'Musica', 'Volume de Vocabulario', 'Diversidade Lexical']]


def words_before_slash_r(text):
    linhas = text.strip().split('\n')
    ultimas_palavras = [linha.split()[-1] for linha in linhas if linha.split()]
    return ultimas_palavras

df['rhymes'] = df['Letra'].apply(words_before_slash_r)
df['rhymes']

df.rename(columns={'index': 'Genero'}, inplace = True)
df.to_excel('lyrical_to_comparison.xlsx', index= False)






In [1]:
import pandas as pd
df = pd.read_excel('lyrical_to_comparison.xlsx')

medias = {}
for genero in df['Genero'].unique():
    filtro = df['Genero'] == genero
    df_filtrado = df[filtro]
    medias[genero] = {
        'Volume de Vocabulario': df_filtrado['Volume de Vocabulario'].mean(),
        'Diversidade Lexical': df_filtrado['Diversidade Lexical'].mean()
    }

# Criar o dicionário
data = {
    'Genero': df['Genero'].unique(),
    'Media Volume de Vocabulário': [medias[genero]['Volume de Vocabulario'] for genero in df['Genero'].unique()],
    'Media Diversidade Lexical': [medias[genero]['Diversidade Lexical'] for genero in df['Genero'].unique()]
}

# Criar o novo DataFrame
novo_df = pd.DataFrame(data)

import plotly.graph_objects as go

fig = go.Figure()

# Adicionar o gráfico de barras
fig.add_trace(go.Bar(
    x=novo_df['Genero'],
    y=novo_df['Media Volume de Vocabulário'],
    text=novo_df['Media Volume de Vocabulário'],
    textposition='auto',  # Define a posição dos valores nos gráficos de barras
    width=0.8  # Ajuste a largura das barras conforme necessário
))

# Personalizar o layout
fig.update_layout(
    title='Média de Volume de Vocabulário por Gênero',
    xaxis=dict(title='Gênero'),
    yaxis=dict(title='Média de Volume de Vocabulário'),
    uniformtext_minsize=8,  # Tamanho mínimo do texto nas barras
    uniformtext_mode='hide',  # Esconde o texto que não cabe
    bargap=0.1  # Ajusta o espaçamento entre as barras
)

# Rotacionar os rótulos do eixo x para melhor legibilidade
fig.update_xaxes(tickangle=90)

# Mostrar o gráfico
fig.show()

fig = go.Figure()

# Adicionar o gráfico de barras
fig.add_trace(go.Bar(
    x=novo_df['Genero'],
    y=novo_df['Media Diversidade Lexical'],
    text=novo_df['Media Diversidade Lexical'],
    textposition='auto',  # Define a posição dos valores nos gráficos de barras
    width=0.8  # Ajuste a largura das barras conforme necessário
))

# Personalizar o layout
fig.update_layout(
    title='Média de Diversidade Lexica',
    xaxis=dict(title='Gênero'),
    yaxis=dict(title='Media Diversidade Lexical'),
    uniformtext_minsize=8,  # Tamanho mínimo do texto nas barras
    uniformtext_mode='hide',  # Esconde o texto que não cabe
    bargap=0.1  # Ajusta o espaçamento entre as barras
)

# Rotacionar os rótulos do eixo x para melhor legibilidade
fig.update_xaxes(tickangle=90)

# Mostrar o gráfico
fig.show()